In [1]:
import pandas as pd
import requests
pd.set_option('display.max_columns', None) 
import time
import numpy as np

URL related to player stats in 2021-22 NBA season

In [2]:
test_url = 'https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=S&Season=2021-22&SeasonType=Regular%20Season&StatCategory=PTS'

obs: based on this url, we can see that the season changes based on the "Season" mark and the season type on the "SeasonType" mark

In [3]:
r = requests.get(url=test_url).json()

Finding the headers names

In [10]:
table_headers = r['resultSet']['headers']

Finding the players stats of the 2018-19 regular season

In [27]:
temp_df1 = pd.DataFrame(r['resultSet']['rowSet'], columns = table_headers)
temp_df1.head()

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
0,1629027,1,Trae Young,1610612737,ATL,76,2652,711,1544,0.460,233,610,0.382,500,553,0.904,50,234,284,737,72,7,303,128,2155,2066,2.43,0.24
1,201942,2,DeMar DeRozan,1610612741,CHI,76,2743,774,1535,0.504,50,142,0.352,520,593,0.877,56,336,392,374,68,24,181,178,2118,1961,2.07,0.38
2,203954,3,Joel Embiid,1610612755,PHI,68,2296,666,1334,0.499,93,251,0.371,654,803,0.814,146,650,796,284,77,99,214,181,2079,2304,1.33,0.36
3,1628369,4,Jayson Tatum,1610612738,BOS,76,2731,708,1564,0.453,230,651,0.353,400,469,0.853,85,524,609,334,75,49,217,174,2046,1971,1.54,0.35
4,203999,5,Nikola Jokic,1610612743,DEN,74,2476,764,1311,0.583,97,288,0.337,379,468,0.810,206,813,1019,584,109,63,281,191,2004,2862,2.08,0.39


Creating columns to indentify the year and the season's type

In [28]:
temp_df2 = pd.DataFrame({'Year': ['2018-19' for i in range(len(temp_18_19))],
             'Season_Type': ['Regular%20Season' for i in range (len(temp_18_19))]})
temp_df2.head()

,Year,Season_Type
0,2018-19,Regular%20Season
1,2018-19,Regular%20Season
2,2018-19,Regular%20Season
3,2018-19,Regular%20Season
4,2018-19,Regular%20Season


In [24]:
temp_df3 = pd.concat([temp_df2, temp_df1], axis=1)
temp_df3.head()

,Year,Season_Type,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
0,2018-19,Regular%20Season,1629027,1,Trae Young,1610612737,ATL,76,2652,711,1544,0.460,233,610,0.382,500,553,0.904,50,234,284,737,72,7,303,128,2155,2066,2.43,0.24
1,2018-19,Regular%20Season,201942,2,DeMar DeRozan,1610612741,CHI,76,2743,774,1535,0.504,50,142,0.352,520,593,0.877,56,336,392,374,68,24,181,178,2118,1961,2.07,0.38
2,2018-19,Regular%20Season,203954,3,Joel Embiid,1610612755,PHI,68,2296,666,1334,0.499,93,251,0.371,654,803,0.814,146,650,796,284,77,99,214,181,2079,2304,1.33,0.36
3,2018-19,Regular%20Season,1628369,4,Jayson Tatum,1610612738,BOS,76,2731,708,1564,0.453,230,651,0.353,400,469,0.853,85,524,609,334,75,49,217,174,2046,1971,1.54,0.35
4,2018-19,Regular%20Season,203999,5,Nikola Jokic,1610612743,DEN,74,2476,764,1311,0.583,97,288,0.337,379,468,0.810,206,813,1019,584,109,63,281,191,2004,2862,2.08,0.39


In [29]:
del temp_df1, temp_df2, temp_df3

In [30]:
df_cols = ['Year', 'Season_Type'] + table_headers

Creating headers so that we can avoid blocking by mimicking browser

In [47]:
headers = {'accept': '*/*'
,'accept-encoding': 'gzip, deflate, br'
,'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7'
,'origin': 'https://www.nba.com'
,'referer': 'https://www.nba.com/'
,'sec-ch-ua': '"Chromium";v="110", "Not A(Brand";v="24", "Google Chrome";v="110"'
,'sec-ch-ua-mobile': '?0'
,'sec-ch-ua-platform': "Linux"
,'sec-fetch-dest': 'empty'
,'sec-fetch-mode': 'cors'
,'sec-fetch-site': 'same-site'
,'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'}

Creating a loop to iterate thorugh nba seasons and between each year's regular season and playoffs stats and saving it into an excel file

In [86]:
df = pd.DataFrame(columns=df_cols)
season_types = ['Regular%20Season', 'Playoffs']
years = ['20' + str(i) + '-' + str(i+1) for i in range(12, 23)]
begin = time.time()

for y in years:
    for s in season_types: 
        api_url = 'https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=S&Season='+ y + '&SeasonType='+ s + '&StatCategory=PTS'
        r = requests.get(url = api_url, headers = headers).json()
        temp_df1 = pd.DataFrame(r['resultSet']['rowSet'], columns = table_headers)
        temp_df2 = pd.DataFrame({'Year': [y for i in range(len(temp_df1))],
                     'Season_Type': [s for i in range (len(temp_df1))]})
        temp_df3 = pd.concat([temp_df2, temp_df1], axis=1)
        df = pd.concat([df, temp_df3], axis=0)
        print(f"{s} from {y} done!")
        lag = np.random.uniform(low=5, high=35)
        print(f"waiting {round(lag, 1)} seconds...")
        time.sleep(lag) # increases chances of making the website accept our scraping

print(f"Processed completed! Total run time: {round((time.time() - begin)/60, 2)}")
df.to_excel('nba_player_data.xlsx', index=False)

Regular%20Season from 2012-13 done!
waiting 13.7 seconds...
Playoffs from 2012-13 done!
waiting 30.9 seconds...
Regular%20Season from 2013-14 done!
waiting 24.2 seconds...
Playoffs from 2013-14 done!
waiting 12.6 seconds...
Regular%20Season from 2014-15 done!
waiting 18.7 seconds...
Playoffs from 2014-15 done!
waiting 28.1 seconds...
Regular%20Season from 2015-16 done!
waiting 27.3 seconds...
Playoffs from 2015-16 done!
waiting 24.7 seconds...
Regular%20Season from 2016-17 done!
waiting 30.3 seconds...
Playoffs from 2016-17 done!
waiting 33.3 seconds...
Regular%20Season from 2017-18 done!
waiting 18.7 seconds...
Playoffs from 2017-18 done!
waiting 18.8 seconds...
Regular%20Season from 2018-19 done!
waiting 21.8 seconds...
Playoffs from 2018-19 done!
waiting 15.1 seconds...
Regular%20Season from 2019-20 done!
waiting 20.0 seconds...
Playoffs from 2019-20 done!
waiting 19.3 seconds...
Regular%20Season from 2020-21 done!
waiting 8.2 seconds...
Playoffs from 2020-21 done!
waiting 13.4 seco